In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("api_key")
os.environ["COHERE_API_KEY"] = api_key

In [7]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

In [8]:
import  pandas as pd

# Load the dataset to a dataframe
#pd.set_option('display.max_colwidth', None)
df_orig = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/atis_intents_train.csv',names=('intent','query'))
#display(df_orig)
sample_classes = ['atis_airfare', 'atis_airline', 'atis_ground_service']
df = df_orig.sample(frac=0.1, random_state=30)
df = df[df.intent.isin(sample_classes)]
df_orig = df_orig.drop(df.index)
df.reset_index(drop=True,inplace=True)

# Remove unnecessary column 
intents = df['intent'] #save for a later need
df.drop(columns=['intent'], inplace=True)
display(df)


,query
0,which airlines fly from boston to washington ...
1,show me the airlines that fly between toronto...
2,show me round trip first class tickets from n...
3,i'd like the lowest fare from denver to pitts...
4,show me a list of ground transportation at bo...
...,...
86,what ground transportation is there in atlanta
87,can i take a single airline from la to charlo...
88,what is the cost for a one way trip from pitt...
89,what ground transportation is available in ba...


In [12]:
from langchain_core.vectorstores import InMemoryVectorStore
documents = df['query'].tolist()
vectorstore = InMemoryVectorStore.from_texts(
    documents,
    embedding=embeddings,
)
# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()
# Retrieve the most similar text
retrieved_documents = retriever.invoke("How can I find a taxi or a bus when the plane lands?")

# show the retrieved document's content
for document in retrieved_documents:
    # Print the content of each document
    print(document.page_content)

 is there ground transportation in boston from the airport
 show me a list of ground transportation at boston airport
 does the phoenix airport have ground transportation to and from downtown
 show me the airlines
